In [1]:
%load_ext autoreload
%load_ext memory_profiler 
%load_ext dotenv
%autoreload 2
%dotenv

In [2]:
from lightning import Trainer
from torchvision.transforms import v2 as T # type: ignore
from geovision.logging import get_logger
from geovision.config.basemodels import ExperimentConfig # noqa
from geovision.data.module import ImageDatasetDataModule
from geovision.training.module import ClassificationModule
from geovision.training.loggers import get_csv_logger, get_wandb_logger, get_ckpt_logger, get_classification_logger

In [3]:
logger = get_logger("experiment_logger")
transforms: dict[str, T.Transform | None] = {
    "image_transform": None,
    "target_transform": None,
    "common_transform": None,
}
config = ExperimentConfig.from_yaml("config.yaml", transforms)
datamodule = ImageDatasetDataModule(config)

In [9]:
litmodule = ClassificationModule(ExperimentConfig.from_yaml("config.yaml", transforms))

loggers: list = list()
callbacks: list = list()
callbacks.append(metric_logger := get_classification_logger(config))
loggers.append(csv_logger := get_csv_logger(config))
# loggers.append(wandb_logger := get_wandb_logger(config))
callbacks.append(ckpt_logger := get_ckpt_logger(config))

# wandb_logger.watch(litmodule)
trainer = Trainer(
    logger = loggers,
    callbacks = callbacks,
    max_epochs = 3,
    check_val_every_n_epoch = 1,
    num_sanity_val_steps = 0,
    #limit_val_batches=1
)

trainer.fit(
    model = litmodule,
    datamodule = datamodule
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/sambhav/miniconda3/envs/dev/lib/python3.12/site-packages/lightning/fabric/loggers/csv_logs.py:268: Experiment logs directory /home/sambhav/experiments/imagenette_classification/can_imagenette_work_with_grace exists and is not empty. Previous log files in this directory will be deleted when the new ones are saved!
/home/sambhav/miniconda3/envs/dev/lib/python3.12/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:653: Checkpoint directory /home/sambhav/experiments/imagenette_classification/can_imagenette_work_with_grace/ckpts exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type                   | Params
--------------------------------------------------------
0 | model        | AlexNet                | 57.0 M
1 | criterion    | CrossEntropyLoss       | 0     
2 | train_metric | Mult

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

0 147


,precision,recall,iou,f1,support
class_name,,,,,
tench,0.805031,0.948148,0.771084,0.870748,135
english_springer,0.880282,0.925926,0.822368,0.902527,135
cassette_player,0.914062,0.866667,0.801370,0.889734,135
chain_saw,0.774775,0.693548,0.577181,0.731915,124
church,0.928000,0.859259,0.805556,0.892308,135
french_horn,0.880952,0.822222,0.740000,0.850575,135
garbage_truck,0.810127,0.948148,0.775758,0.873720,135
gas_pump,0.830769,0.800000,0.687898,0.815094,135
golf_ball,0.926829,0.844444,0.791667,0.883721,135


...

,precision,recall,iou,f1,support
class_name,,,,,
tench,0.900709,0.940741,0.852349,0.920290,135
english_springer,0.961538,0.925926,0.892857,0.943396,135
cassette_player,0.872340,0.911111,0.803922,0.891304,135
chain_saw,0.792308,0.830645,0.682119,0.811024,124
church,0.930233,0.888889,0.833333,0.909091,135
french_horn,0.909091,0.814815,0.753425,0.859375,135
garbage_truck,0.913043,0.933333,0.857143,0.923077,135
gas_pump,0.805556,0.859259,0.711656,0.831541,135
golf_ball,0.945312,0.896296,0.852113,0.920152,135


Validation: |          | 0/? [00:00<?, ?it/s]

2 441


,precision,recall,iou,f1,support
class_name,,,,,
tench,0.909091,0.962963,0.878378,0.935252,135
english_springer,0.909091,0.962963,0.878378,0.935252,135
cassette_player,0.872483,0.962963,0.844156,0.915493,135
chain_saw,0.841270,0.854839,0.736111,0.848000,124
church,0.917293,0.903704,0.835616,0.910448,135
french_horn,0.902256,0.888889,0.810811,0.895522,135
garbage_truck,0.939394,0.918519,0.867133,0.928839,135
gas_pump,0.883721,0.844444,0.760000,0.863636,135
golf_ball,0.967742,0.888889,0.863309,0.926641,135


`Trainer.fit` stopped: `max_epochs=3` reached.


In [ ]:
from pathlib import Path
import h5py
import torch
import torchmetrics
import numpy as np
import pandas as pd


logs_dir = Path.home()/"experiments"/"imagenette_classification"/'_'.join(config.name.split(' '))/"val_logs"
logits_file = sorted(logs_dir.rglob("*.h5"))[-1]
display(logits_file)
with h5py.File(logits_file) as f:
    logits = torch.tensor(f["logits"][:])
    labels = torch.tensor(f["labels"][:])
    df_idxs = f["df_idxs"][:]

# torchmetrics.functional.confusion_matrix(logits, labels, "multiclass", num_classes=10)
# pd.Series(logits.argmax(1).tolist()).value_counts()
def get_confusion_matrix(logits, labels):
    num_classes = logits.shape[1] 
    confm = np.zeros((num_classes, num_classes), dtype = np.uint64)
    for i in range(len(logits)):
        pred_idx = np.argmax(logits[i])
        true_idx = labels[i]
        confm[true_idx, pred_idx] += 1
    display(confm)

get_confusion_matrix(logits, labels)

In [ ]:
# multilabel classification output
logits = torch.randn(5, dtype = torch.float32)
probs = torch.sigmoid(logits)
truth = torch.tensor([0, 1, 0, 1, 0], dtype = torch.float32)

print(probs, probs.sum())
print(truth, truth.sum())

torch.nn.BCEWithLogitsLoss()(truth, probs)